## Deliverable 2. Create a Customer Travel Destinations Map.

In [62]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import matplotlib.pyplot as plt
# Import API key
from config import g_key
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [16]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity (%),Cloudiness (%),Wind Speed,Current Description
0,0,Kamenka,RU,51.3223,42.7678,71.92,57,12,9.01,few clouds
1,1,Yaypan,UZ,40.3758,70.8156,88.57,30,100,11.23,overcast clouds
2,2,Arraial Do Cabo,BR,-22.9661,-42.0278,66.31,85,100,4.76,overcast clouds
3,3,Upernavik,GL,72.7868,-56.1549,24.73,90,100,13.87,overcast clouds
4,4,Busselton,AU,-33.6500,115.3333,59.20,70,27,27.96,scattered clouds


In [17]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [18]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity (%),Cloudiness (%),Wind Speed,Current Description
0,0,Kamenka,RU,51.3223,42.7678,71.92,57,12,9.01,few clouds
6,6,Grand Gaube,MU,-20.0064,57.6608,73.80,83,100,5.01,light rain
9,9,Atuona,PF,-9.8000,-139.0333,78.03,66,11,14.07,few clouds
10,10,Pasighat,IN,28.0667,95.3333,77.40,94,97,3.69,moderate rain
11,11,Tual,ID,-5.6667,132.7500,82.81,76,100,10.87,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
661,661,Akdepe,TM,42.0551,59.3788,84.29,21,8,6.08,clear sky
664,664,Sargur,IN,11.9833,76.4167,82.98,54,95,12.66,overcast clouds
665,665,Raga,SS,8.4596,25.6780,82.80,59,84,5.26,broken clouds
666,666,Vanavara,RU,60.3400,102.2797,71.15,30,100,4.07,overcast clouds


In [19]:
# 4a. Determine if there are any empty rows.

print(preferred_cities_df.count())
print(preferred_cities_df.isnull().sum())


City_ID                278
City                   278
Country                278
Lat                    278
Lng                    278
Max Temp               278
Humidity (%)           278
Cloudiness (%)         278
Wind Speed             278
Current Description    278
dtype: int64
City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity (%)           0
Cloudiness (%)         0
Wind Speed             0
Current Description    0
dtype: int64


In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

#No Empty Row found in 4a, thus moving to 5a.

In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kamenka,RU,71.92,few clouds,51.3223,42.7678,
6,Grand Gaube,MU,73.80,light rain,-20.0064,57.6608,
9,Atuona,PF,78.03,few clouds,-9.8000,-139.0333,
10,Pasighat,IN,77.40,moderate rain,28.0667,95.3333,
11,Tual,ID,82.81,overcast clouds,-5.6667,132.7500,
14,Bambous Virieux,MU,73.65,moderate rain,-20.3428,57.7575,
19,Kapaa,US,80.58,broken clouds,22.0752,-159.3190,
22,Bengkulu,ID,82.96,overcast clouds,-3.8004,102.2655,
24,Butaritari,KI,82.29,few clouds,3.0707,172.7902,
26,Kisangani,CD,71.67,moderate rain,0.5167,25.2000,


In [32]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522",
    "language": "English"}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame

    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotel=requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found... Skipping {index}")
        

Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping


In [54]:
# 7. Drop the rows where there is no Hotel Name.

clean_hotel_df = hotel_df.replace(r'^\s*$', np.nan, regex = True).dropna()

clean_hotel_df


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Grand Gaube,MU,73.80,light rain,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
9,Atuona,PF,78.03,few clouds,-9.8000,-139.0333,Villa Enata
10,Pasighat,IN,77.40,moderate rain,28.0667,95.3333,Siang Guest House
11,Tual,ID,82.81,overcast clouds,-5.6667,132.7500,Grand Vilia Hotel
14,Bambous Virieux,MU,73.65,moderate rain,-20.3428,57.7575,Casa Tia Villa
...,...,...,...,...,...,...,...
649,Indramayu,ID,84.40,overcast clouds,-6.3264,108.3200,Grand Trisula Hotel
653,Tura,IN,83.52,moderate rain,25.5198,90.2201,Hotel RIKMAN Continental
654,Fomboni,KM,79.70,clear sky,-12.2800,43.7425,Auberge les abou
664,Sargur,IN,82.98,overcast clouds,11.9833,76.4167,Sri rama jyothishyalaya


In [56]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [57]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Current Description}
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [63]:
# 11a. Add a marker layer for each city to the map. 

max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>